Reconfiguring Source to parallelize into dask geodataframes. Currently Testing the Flood ZOne split and BFE cleaning process. Many of the 
functions may not work in a dask df. more research is required. however, if this works, this will be 10x faster than using conventional 
geodataframes. 

global structure of iterating through County GDBs should mimic final structure once "CO_Merging_Scaleup_A" is finalized. 

In [1]:
import dask.dataframe as d
import dask_geopandas as dd
import glob
from new_methods import *

In [24]:
select_gdb = '48113'
for gdb in glob.glob('D:\Projects\WaterSurfacePolygons\Data\CountyEdge_Testing\*.gdb'):
    if os.path.basename(gdb).split('.')[0] == select_gdb:
        selFZ = gpd.read_file(gdb, driver='FileGDB', layer='FLOOD_ZONES')
        selFZ = dd.from_geopandas(selFZ, npartitions=4)

        selBFE = gpd.read_file(gdb, driver='FileGDB', layer='DYNAMIC')
        selBFE = dd.from_geopandas(selBFE, npartitions=4)

        sel_ae = selFZ.loc[selFZ['D_ZONE'] == 'AE']
        sel_ae_dynamic = sel_ae.loc[sel_ae['BFE_STATIC'].isnull()]
        sel_ae_dynamic['FIPS'] = sel_ae_dynamic.apply(lambda x: str(x['STATE_CODE']) + str(x['COUNTY_CODE']), axis=1)

        sel_ae_dis = sel_ae_dynamic.dissolve(by='FIPS')
        sel_ae_dis = sel_ae_dis.to_crs(26913)
        sel_fsp = sel_ae_dis[['geometry']]
        sel_fsp = sel_fsp.reset_index()

        # Reproject BFEs 
        selBFE = selBFE.to_crs(26913)
        sel_bfe = selBFE[['ELEV', 'geometry']]

        """ Test List of adjacent Co FIPS """
        adj_ls = ['48439'] # Only tarrant
        for a in adj_ls:
            for adj_gdb in glob.glob('D:\Projects\WaterSurfacePolygons\Data\CountyEdge_Testing\*.gdb'):
                if os.path.basename(adj_gdb).split('.')[0] == a:
                    adjFZ = gpd.read_file(adj_gdb, driver='FileGDB', layer='FLOOD_ZONES')
                    adjFZ = dd.from_geopandas(adjFZ, npartitions=4)

                    adjBFE = gpd.read_file(adj_gdb, driver='FileGDB', layer='DYNAMIC')
                    adjBFE = dd.from_geopandas(adjBFE, npartitions=4)

                    adj_ae = adjFZ.loc[adjFZ['D_ZONE'] == 'AE']
                    adj_ae_dynamic = adj_ae.loc[adj_ae['BFE_STATIC'].isnull()]
                    adj_ae_dynamic['FIPS'] = adj_ae_dynamic.apply(lambda x: str(x['STATE_CODE']) + str(x['COUNTY_CODE']), axis=1)

                    adj_ae_dis = adj_ae_dynamic.dissolve(by='FIPS')
                    adj_ae_dis = adj_ae_dis.to_crs(26913)
                    adj_fsp = adj_ae_dis[['geometry']]
                    adj_fsp = adj_fsp.reset_index()

                    # Reproject BFEs 
                    adjBFE = adjBFE.to_crs(26913)
                    adj_bfe = adjBFE[['ELEV', 'geometry']]

        print('Merge All County Flood Zones')
        all_fsp = d.concat([sel_fsp, adj_fsp], ignore_index=True)
        all_fsp_x = all_fsp.explode()
        all_fsp_merge = dd.from_geopandas(g(all_fsp_x.unary_union.compute(), 26913), npartitions=4)

        print('Merge All County BFEs')
        all_bfe = d.concat([sel_bfe, adj_bfe], ignore_index=True)

        print('Clean BFEs')
        bfe_bfe = all_bfe.sjoin(all_bfe)
        bfe_bfe = bfe_bfe.reset_index()
        brok = bfe_bfe.loc[bfe_bfe['index'] != bfe_bfe['index_right']]
        if len(brok) > 0:
            print('Cleaning BFEs...')
            brok_geoms = brok[['index', 'geometry']]
            fix = brok.merge(brok_geoms, left_on='index_right', right_on='index')

            # linemerge([m for m in x.geometry_x] + [m for m in x.geometry_y]
            fix['new_geom'] = fix.apply(lambda x: linemerge(list((x.geometry_x, x.geometry_y)))
                                         if (x.geometry_x.geom_type == 'LineString' and  
                                        x.geometry_y.geom_type == 'LineString') 
                                        else linemerge([m for m in x.geometry_x] + [m for m in x.geometry_y]), 
                                        axis=1)

            fix_gpd = fix[['index_x', 'ELEV_left', 'new_geom']]
            fix_gpd = fix_gpd.set_geometry('new_geom')
            fix_gpd = fix_gpd.rename(columns={'index_x': 'index', 'ELEV_left': 'ELEV', 'new_geom': 'geometry'})
            fix_gpd = fix_gpd.set_index('index')
            bfe_clean = all_bfe.loc[~all_bfe.index.isin(fix_gpd.index)]
            bfe = d.concat([bfe_clean, fix_gpd])
            bfe = bfe.drop_duplicates('geometry')

        # Extend BFEs over FSP Poly then reset bfe to new position
        print('Extending BFE...')
        bfe_ext = bfe_extend(bfe, all_fsp_merge)

        print('Clipping BFE...')
        bfe = bfe_ext.overlay(all_fsp_merge)

        print('Splitting FSP...')
        # Split FSP Poly by extended BFEs
        fsp_s = split_fsp(all_fsp_merge, bfe_ext)

        print('Cleaning FSP...')
        # Remove slivers due to overlaps from extended BFEs
        fsp_s = fsp_s.overlay(all_fsp_merge)
        fsp_s = fsp_s.loc[fsp_s.geom_type == 'Polygon']
        fsp_s = fsp_s[['geometry']]
        fsp_s = fsp_s.reset_index()

Merge All County Flood Zones
Merge All County BFEs
Clean BFEs
Cleaning BFEs...


ValueError: Unknown column new_geom

In [12]:
all_fsp_merge = dd.from_geopandas(g(all_fsp_x.unary_union.compute(), 26913), npartitions=4)

In [14]:
all_fsp_merge.to_feather('all_fsp.feather')

(None,)

In [15]:
""" I/O Scramble to take dask geodataframe into a Shapefile """
for fi in glob.glob('all_fsp.feather/*.feather'):
    df = gpd.read_feather(os.path.abspath(fi))
    n = os.path.basename(fi)
    df.to_file(f'{n}.shp')